In [4]:
import pandas as pd
import numpy as np 
from datetime import datetime
import os
import re

In [3]:
plates = r'5_Decant/'

In [34]:
SOL = []
INSOL = []
SOLPLATES = {}
INSOLPLATES = {}

for file in os.listdir(plates):
            if file.endswith('.log'):
                PATH = os.path.join(plates, file)
                f1 = open(PATH, 'r').readlines()
                DILUTIONPLATE = [item for item in f1 if re.search(r'D[0-9]{1,9}', item)]
                for item in DILUTIONPLATE:
                        META = item.split(',')
                        DIL = META[5]
                        PLA = META[4]
                        TIME = datetime.strptime(META[0], '%m/%d/%Y %H:%M:%S')
                        if PLA not in (SOL and INSOL):
                            if re.search(r'S', PLA) != None:
                                    SOL.append(PLA)
                                    try:
                                        if SOLPLATES.get(DIL) != PLA:
                                            SOLPLATES.update({DIL:PLA})
                                        else:
                                            continue
                                    except:
                                        SOLPLATES.update({DIL:PLA})
                            
                            if re.search(r'I', PLA) != None:
                                    INSOL.append(PLA)
                                    try:
                                        if INSOLPLATES.get(DIL) != PLA:
                                            INSOLPLATES.update({DIL:PLA})
                                        else:
                                            continue
                                    except:
                                        INSOLPLATES.update({DIL:PLA})
                        else:
                            continue
                

In [35]:
SOLPLATES

{'D0000002': 'S0000002', 'D0000010': 'S0000010'}

In [36]:
INSOLPLATES

{'D0000002': 'I0000002', 'D0000010': 'I0000010'}

In [12]:
PLATE_DATA = []
DIR = r'OneDrive_2023-09-19/Peak Tables/'

# Initialize an empty dictionary to store the nested structure
nested_dict = {}

for file in os.listdir(DIR):
    if file.endswith('.csv'):
        PATH = os.path.join(DIR, file)
        f1 = pd.read_csv(PATH)
        f1['Reduced_Plate'] = f1['Plate Name'].apply(lambda x: re.search(r'[A-Z]{1,9}[0-9]{1,9}', x).group(0))
        for row, index in f1.iterrows():
            key = index['Reduced_Plate']
            sub_key = index['Sample Name']
            value = (index['Conc. (ng/ul)'], file)
            # Check if the outer key exists in the nested_dict
            if key in nested_dict:
                nested_dict[key][sub_key] = value
            else:
                nested_dict[key] = {sub_key: value}
        PLATE_DATA.append(f1)

PLATE_DATA = pd.concat(PLATE_DATA, axis=0, ignore_index=True)
PLATE_DATA.head(n=5)

,Well Number,Sample Name,Well Label,Migration Time(sec) - Center,Peak Number,Height,Corr. Area,Expected Protein [kDa],Size [kDa],Conc. (ng/ul),% Purity,Type,FWHM (sec),Comment,Plate Name,Reduced_Plate
0,2,A1,A01,29.902573,41,18.171939,22.117497,150,153.552518,8.219369,0.788827,EP150,0.269695,NaN,RED0000012_2023-09-11_11-53-19,RED0000012
1,7,A6,A06,29.449489,44,12.437826,12.021422,150,146.610274,5.042328,1.141297,EP150,0.209525,NaN,RED0000012_2023-09-11_11-53-19,RED0000012
2,8,A7,A07,29.417011,46,11.669323,11.170058,150,146.115883,4.836147,1.035069,EP150,0.202330,NaN,RED0000012_2023-09-11_11-53-19,RED0000012
3,10,A9,A09,29.902010,45,12.885283,11.762757,150,153.543838,5.314913,0.596501,EP150,0.193818,NaN,RED0000012_2023-09-11_11-53-19,RED0000012
4,11,A10,A10,29.848359,41,295.054164,118.888745,150,152.717501,53.760758,5.867834,EP150,0.073671,NaN,RED0000012_2023-09-11_11-53-19,RED0000012


In [13]:
nested_dict

{'RED0000012': {'A1': (8.21936898690468,
   'RED0000012_2023-09-11_11-53-19_PeakTable.csv'),
  'A6': (5.04232783245498, 'RED0000012_2023-09-11_11-53-19_PeakTable.csv'),
  'A7': (4.83614688329577, 'RED0000012_2023-09-11_11-53-19_PeakTable.csv'),
  'A9': (5.31491286910658, 'RED0000012_2023-09-11_11-53-19_PeakTable.csv'),
  'A10': (53.7607579135201, 'RED0000012_2023-09-11_11-53-19_PeakTable.csv'),
  'A11': (5.31966626800305, 'RED0000012_2023-09-11_11-53-19_PeakTable.csv'),
  'B1': (8.00901855016116, 'RED0000012_2023-09-11_11-53-19_PeakTable.csv'),
  'B2': (4.6049771141276, 'RED0000012_2023-09-11_11-53-19_PeakTable.csv'),
  'B9': (6.15874372215905, 'RED0000012_2023-09-11_11-53-19_PeakTable.csv'),
  'B10': (7.94418020911716, 'RED0000012_2023-09-11_11-53-19_PeakTable.csv'),
  'C1': (13.2281036620326, 'RED0000012_2023-09-11_11-53-19_PeakTable.csv'),
  'C2': (5.75209559579097, 'RED0000012_2023-09-11_11-53-19_PeakTable.csv'),
  'C5': (6.59550866935967, 'RED0000012_2023-09-11_11-53-19_PeakTable.

In [25]:
def get_peak_data(plate, well):
    try:
        return nested_dict[plate][well][0]
    except KeyError:
        return ("None")
    
def get_peak_log(plate, well):
    try:
        return nested_dict[plate][well][1]
    except KeyError:
        return ("None")

In [5]:
META = pd.read_csv(r'Consolidated.csv')
META.head(n=5)

/var/folders/16/jyj8jxwj4dq3x0bkzzp46ljc0000gn/T/ipykernel_23063/740772304.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  META = pd.read_csv(r'Consolidated.csv')


,ECHO_Source Plate Name,ECHO_Source Plate Barcode,ECHO_Source Well,384_AGAR_PLATE,echo_well_pick_actionTime,384_AGAR_PLATE_WELL,96_GROWTH_PLATE,Sample Name,log_file,EchoCherryPick_well_pick_actionTime,...,Dilution_Autiomation_Serial_Number,Glycerol_Plate,Glycerol_Autiomation_Serial_Number,Insol_Plate,Sol_Plate,Sol_Reduced_CGEPlate,Insol_Reduced_CGEPlate,Sol Conc. (ng/ul),Insol Conc. (ng/ul),Peak Log
0,Source Plate[1],Source[1],B1,A0000404,09/05/2023 16:19:40,O6,G0000426,B7,UnifiedTransfer09-05-2023 16.18.38.log,2023-09-05 16:19:40,...,B8758519K13,GLY0000002,B8758519K13,I0000002,S0000002,RED0000008,RED0000024,NaN,NaN,NaN
1,Source Plate[1],Source[1],B1,A0000404,09/05/2023 16:19:40,M6,G0000426,C7,UnifiedTransfer09-05-2023 16.18.38.log,2023-09-05 16:19:40,...,B8758519K13,GLY0000002,B8758519K13,I0000002,S0000002,RED0000008,RED0000024,5.655277,NaN,RED0000008_2023-09-07_15-10-58_PeakTable.csv
2,Source Plate[1],Source[1],B1,A0000404,09/05/2023 16:19:40,J6,G0000426,D7,UnifiedTransfer09-05-2023 16.18.38.log,2023-09-05 16:19:40,...,B8758519K13,GLY0000002,B8758519K13,I0000002,S0000002,RED0000008,RED0000024,NaN,NaN,NaN
3,Source Plate[1],Source[1],B1,A0000404,09/05/2023 16:19:40,O5,G0000426,E7,UnifiedTransfer09-05-2023 16.18.38.log,2023-09-05 16:19:40,...,B8758519K13,GLY0000002,B8758519K13,I0000002,S0000002,RED0000008,RED0000024,NaN,NaN,NaN
4,Source Plate[1],Source[1],B1,A0000404,09/05/2023 16:19:40,L4,G0000426,F7,UnifiedTransfer09-05-2023 16.18.38.log,2023-09-05 16:19:40,...,B8758519K13,GLY0000002,B8758519K13,I0000002,S0000002,RED0000008,RED0000024,NaN,NaN,NaN


In [31]:
len(META)

244992

In [32]:
META["Sol Conc. (ng/ul)"] = META.apply(lambda row: get_peak_data(row['Sol_Reduced_Plate'], row['Sample Name']), axis=1)
META["Insol Conc. (ng/ul)"] = META.apply(lambda row: get_peak_data(row['Insol_Reduced_Plate'], row['Sample Name']), axis=1)

META["Peak Log"] = META.apply(lambda row: get_peak_log(row['Sol_Reduced_Plate'], row['Sample Name']), axis=1)

In [35]:
META.head(n=5)

,ECHO_Source Plate Name,ECHO_Source Plate Barcode,ECHO_Source Well,384_AGAR_PLATE,echo_well_pick_actionTime,384_AGAR_PLATE_WELL,96_GROWTH_PLATE,Sample Name,log_file,EchoCherryPick_well_pick_actionTime,...,Dilution_Autiomation_Serial_Number,Glycerol_Plate,Glycerol_Autiomation_Serial_Number,Insol_Plate,Sol_Plate,Sol_Reduced_Plate,Insol_Reduced_Plate,Sol Conc. (ng/ul),Insol Conc. (ng/ul),Peak Log
0,Source Plate[1],Source[1],B1,A0000404,09/05/2023 16:19:40,O6,G0000426,B7,UnifiedTransfer09-05-2023 16.18.38.log,2023-09-05 16:19:40,...,B8758519K13,GLY0000002,B8758519K13,I0000002,S0000002,RED0000008,RED0000024,None,None,None
1,Source Plate[1],Source[1],B1,A0000404,09/05/2023 16:19:40,M6,G0000426,C7,UnifiedTransfer09-05-2023 16.18.38.log,2023-09-05 16:19:40,...,B8758519K13,GLY0000002,B8758519K13,I0000002,S0000002,RED0000008,RED0000024,5.655277,None,RED0000008_2023-09-07_15-10-58_PeakTable.csv
2,Source Plate[1],Source[1],B1,A0000404,09/05/2023 16:19:40,J6,G0000426,D7,UnifiedTransfer09-05-2023 16.18.38.log,2023-09-05 16:19:40,...,B8758519K13,GLY0000002,B8758519K13,I0000002,S0000002,RED0000008,RED0000024,None,None,None
3,Source Plate[1],Source[1],B1,A0000404,09/05/2023 16:19:40,O5,G0000426,E7,UnifiedTransfer09-05-2023 16.18.38.log,2023-09-05 16:19:40,...,B8758519K13,GLY0000002,B8758519K13,I0000002,S0000002,RED0000008,RED0000024,None,None,None
4,Source Plate[1],Source[1],B1,A0000404,09/05/2023 16:19:40,L4,G0000426,F7,UnifiedTransfer09-05-2023 16.18.38.log,2023-09-05 16:19:40,...,B8758519K13,GLY0000002,B8758519K13,I0000002,S0000002,RED0000008,RED0000024,None,None,None


In [34]:
META[META['Sol Conc. (ng/ul)'] != "None"].head(n=5)

,ECHO_Source Plate Name,ECHO_Source Plate Barcode,ECHO_Source Well,384_AGAR_PLATE,echo_well_pick_actionTime,384_AGAR_PLATE_WELL,96_GROWTH_PLATE,Sample Name,log_file,EchoCherryPick_well_pick_actionTime,...,Dilution_Autiomation_Serial_Number,Glycerol_Plate,Glycerol_Autiomation_Serial_Number,Insol_Plate,Sol_Plate,Sol_Reduced_Plate,Insol_Reduced_Plate,Sol Conc. (ng/ul),Insol Conc. (ng/ul),Peak Log
1,Source Plate[1],Source[1],B1,A0000404,09/05/2023 16:19:40,M6,G0000426,C7,UnifiedTransfer09-05-2023 16.18.38.log,2023-09-05 16:19:40,...,B8758519K13,GLY0000002,B8758519K13,I0000002,S0000002,RED0000008,RED0000024,5.655277,None,RED0000008_2023-09-07_15-10-58_PeakTable.csv
7,Source Plate[1],Source[1],B1,A0000404,09/05/2023 16:20:10,P2,G0000426,A8,UnifiedTransfer09-05-2023 16.18.38.log,2023-09-05 16:20:10,...,B8758519K13,GLY0000002,B8758519K13,I0000002,S0000002,RED0000008,RED0000024,7.049333,None,RED0000008_2023-09-07_15-10-58_PeakTable.csv
9,Source Plate[1],Source[1],B1,A0000404,09/05/2023 16:20:10,M2,G0000426,C8,UnifiedTransfer09-05-2023 16.18.38.log,2023-09-05 16:20:10,...,B8758519K13,GLY0000002,B8758519K13,I0000002,S0000002,RED0000008,RED0000024,32.054411,None,RED0000008_2023-09-07_15-10-58_PeakTable.csv
10,Source Plate[1],Source[1],B1,A0000404,09/05/2023 16:20:10,J2,G0000426,D8,UnifiedTransfer09-05-2023 16.18.38.log,2023-09-05 16:20:10,...,B8758519K13,GLY0000002,B8758519K13,I0000002,S0000002,RED0000008,RED0000024,4.668549,None,RED0000008_2023-09-07_15-10-58_PeakTable.csv
21,Source Plate[1],Source[1],B1,A0000404,09/05/2023 16:37:15,P11,G0000426,A9,UnifiedTransfer09-05-2023 16.18.38.log,2023-09-05 16:37:15,...,B8758519K13,GLY0000002,B8758519K13,I0000002,S0000002,RED0000008,RED0000024,4.634932,None,RED0000008_2023-09-07_15-10-58_PeakTable.csv


In [1]:
import random

# Initialize a list to keep track of used numbers
used_numbers = []

def generate_random_number():
    while True:
        # Generate a random number
        new_number = random.randint(1, 999999)
        
        # Format the number as 'R' followed by a zero-padded 6-digit numeric value
        formatted_number = f'R{new_number:06d}'
        
        # Check if the number has already been used
        if formatted_number not in used_numbers:
            used_numbers.append(formatted_number)
            return formatted_number

# Example usage
for _ in range(10):
    random_number = generate_random_number()
    print(random_number)


R754166
R603478
R901485
R085876
R436752
R249006
R362489
R337591
R945677
R811052


In [7]:
from neo4j import GraphDatabase

In [8]:
# Define the connection details
uri = "bolt://localhost:7687"  # Update this with your Neo4j server URI
username = "neo4j"
password = 'plates1234'

In [15]:
AGGARPLATES = META["384_AGAR_PLATE"].unique().tolist()

In [25]:
def replace_list_property(uri, username, password, node_id, property_name, new_list):
    query = (
        f"MATCH(p:{node_id}) "
        f"SET p.{property_name} = {new_list}"
    )

    with GraphDatabase.driver(uri, auth=(username, password)) as driver:
        with driver.session() as session:
            session.write_transaction(_execute_query, query)

def _execute_query(tx, query):
    tx.run(query)

In [26]:
replace_list_property(uri, username, password, "AgarPlate", "Plates", AGGARPLATES)

/var/folders/16/jyj8jxwj4dq3x0bkzzp46ljc0000gn/T/ipykernel_23063/4203306265.py:9: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(_execute_query, query)


In [41]:
 # Initialize a list to keep track of used numbers
def get_plates_property(node_id, plate_type, session):
    query = (
        f"MATCH (node:{node_id} {{PlateTYPE: '{plate_type}'}}) "
        "RETURN node.Plates AS platesProperty"
    )
    
    result = session.run(query)
    
    # Extract the platesProperty from the result
    for record in result:
        plates_property = record["platesProperty"]
        return plates_property  # Return the "Plates" property value

In [42]:
node_id = "EchoPlate"
plate_type = "Echo Plate"

with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    with driver.session() as session:
        TEST = get_plates_property(node_id, plate_type, session)

In [43]:
print(TEST)

['R754166', 'R603478', 'R901485']
